In [15]:
import pandas as pd
import os.path
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [14]:
# Read and combine mass shooting data files into dataframe
ms_df = pd.concat(
    map(pd.read_csv, ["Resources/Mass_shootings_2019.csv", "Resources/Mass_shootings_2018.csv","Resources/Mass_shootings_2017.csv"]), ignore_index=True)
ms_df.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,1582816,"December 29, 2019",Illinois,Danville,1803 block of Westview Ave,0,5,NaN
1,1582197,"December 29, 2019",New York,Buffalo,50 block of Henrietta Ave,1,3,NaN
2,1583444,"December 29, 2019",California,Ceres,800 block of Allacante Dr,0,5,NaN
3,1581590,"December 27, 2019",California,Modesto,1900 block of Vernon Ave,1,3,NaN
4,1581103,"December 27, 2019",Texas,Houston,500 block of Smart St,2,7,NaN


In [53]:
# Read CSV file
csv_file = "Resources/Population_data_2019.csv"
pop_2019_df = pd.read_csv(csv_file, header=None);

# Remove first 2 rows and make 3rd row header
pop_2019_df = pop_2019_df.iloc[2:,:]
new_header = pop_2019_df.iloc[0]
pop_2019_df = pop_2019_df[1:]
pop_2019_df.columns= new_header

# Remove totals row
pop_2019_df = pop_2019_df.iloc[1:,:]

# New column for year of data
filedate=2019
pop_2019_df[8]=filedate

# Rename and sort columns
pop_2019_df.rename(columns ={"Children 0-18":"0-18",
                            "Adults 19-25":"19-25",
                            "Adults 26-34":"26-34",
                            "Adults 35-54":"35-54",
                            "Adults 55-64":"55-64",
                            }, inplace = True)

pop_2019_df.head()

2,Location,0-18,19-25,26-34,35-54,55-64,65+,Total,8
4,Alabama,1128100,414500,549500,1195300,645000,834700,4767100,2019
5,Alaska,181700,65300,97500,172900,92800,91500,701700,2019
6,Arizona,1701600,656800,860500,1708100,877500,1293600,7098000,2019
7,Arkansas,720800,253500,335600,723700,383200,505700,2922500,2019
8,California,9171300,3539800,5288800,10152200,4751600,5739000,38642700,2019


In [47]:
cols = list(pop_2019_df.columns.values)
print(cols)  

['Location', '0-18', '19-25', '26-34', '35-54', '55-64', '65+', 'Total', 8]


In [52]:
csv_file = "Resources/Population_data_2018.csv"
filedate=2018
pop_2018_df = pd.read_csv(csv_file, header=None);
pop_2018_df[8]=filedate
pop_2018_df = pop_2018_df.iloc[2:,:]
new_header = pop_2018_df.iloc[0]
pop_2018_df = pop_2018_df[1:]
pop_2018_df.columns= new_header

pop_2018_df.rename(columns ={"Children 0-18":"0-18",
                            "Adults 19-25":"19-25",
                            "Adults 26-34":"26-34",
                            "Adults 35-54":"35-54",
                            "Adults 55-64":"55-64",
                            "2018":"Year",
                            }, inplace = True)



pop_2018_df.head()

2,Location,0-18,19-25,26-34,35-54,55-64,65+,Total,2018
3,United States,75877700,28020700,39590700,81990600,41897600,51121200,318498500,2018
4,Alabama,1128300,419500,554400,1192600,646800,811000,4752600,2018
5,Alaska,191100,62200,93400,182800,94100,85500,709100,2018
6,Arizona,1706300,645900,841100,1702900,862200,1245900,7004300,2018
7,Arkansas,728600,255200,345400,722800,380200,489100,2921300,2018


In [36]:
csv_file = "Resources/Population_data_2017.csv"
filedate=2017
pop_2017_df = pd.read_csv(csv_file, header=None);
pop_2017_df[8]=filedate
pop_2017_df = pop_2017_df.iloc[2:,:]
new_header = pop_2017_df.iloc[0]
pop_2017_df = pop_2017_df[1:]
pop_2017_df.columns= new_header
pop_2017_df.head()

2,Location,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,Total,2017
3,United States,76119000,28348600,39097500,82323500,41648400,49485600,317022500,2017
4,Alabama,1138300,430500,536200,1207200,645600,783600,4741400,2017
5,Alaska,192300,63500,97900,183600,92400,80500,710100,2017
6,Arizona,1686200,637200,815200,1677000,839900,1185700,6841300,2017
7,Arkansas,730600,265200,329800,726600,377100,480600,2910000,2017
